In [1]:
%%bash
cd ../mujoco_env
bash generate_darm_xml.sh true true

Running generate_darm_xml.sh
Single Finger: true
No Wrist: true






In [1]:
import numpy as np
import gym
from darm_gym_env import DARMEnv

env = gym.make("darm/DarmHand-v0", render_mode="human", hand_name="hand1",
               single_finger_env=True)

n_fingertips = 1

Loaded XML file successfully


/home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [3]:
def norm_to_target(obs):
    """
    Returns the norm of each fingertip to the target position
    obs: an observation from the observation space [...fingertip_pos, ...target_pos, ...fingertip_vel]
    """
    obs = obs.reshape((-1, 3))

    fingertip_poses = obs[0:n_fingertips]
    target_poses = obs[n_fingertips:2*n_fingertips]

    return np.linalg.norm(fingertip_poses-target_poses, ord=2, axis=-1)


In [4]:
import time

while True:
    reset_start_time = time.time()
    obs = env.reset()
    print(f"Reset took {time.time() - reset_start_time} s")
    
    mocap_pos = np.array(env.data.mocap_pos - env.ref_pos).flatten()
    # print(f"Mocap Equiv: {np.array_equal(mocap_pos, obs[n_fingertips:2*n_fingertips])}")
    
    f1_pos = obs[0*3:1*3]
    f1_mocap = mocap_pos[0*3:1*3]
    dist = np.linalg.norm(f1_pos - f1_mocap)*1000
    
    norm = norm_to_target(obs)
    print(f"Within Limit: {(all(norm >= env.min_target_th) and all(norm <= env.max_target_th))}")
    print(f"Norm to target: {norm*1000} mm\n")
    
    # print((dist, np.max(np.abs(f1_pos - f1_mocap))*1000))
    # print((f1_pos, f1_mocap))
    # print("\n\n")
    if np.max(np.abs(f1_pos - f1_mocap))*1000 > dist: 
        print("Error")
        print((dist, np.max(np.abs(f1_pos - f1_mocap))*1000))
        print((f1_pos, f1_mocap))
        break
    time.sleep(2)

Reset took 0.3878815174102783 s
Within Limit: True
Norm to target: [9.5822755] mm

Reset took 0.04100513458251953 s
Within Limit: True
Norm to target: [9.76511205] mm

Reset took 0.04379105567932129 s
Within Limit: True
Norm to target: [11.68179075] mm

Reset took 0.03560614585876465 s
Within Limit: True
Norm to target: [14.77420238] mm

Reset took 0.010220766067504883 s
Within Limit: True
Norm to target: [10.98750074] mm

Reset took 0.01155996322631836 s
Within Limit: True
Norm to target: [14.43841471] mm

Reset took 0.06046772003173828 s
Within Limit: True
Norm to target: [10.76068748] mm

Reset took 0.01226186752319336 s
Within Limit: True
Norm to target: [12.20147028] mm

Reset took 0.03178858757019043 s
Within Limit: True
Norm to target: [9.77912962] mm

Reset took 0.013421297073364258 s
Within Limit: True
Norm to target: [9.50664191] mm

Reset took 0.01794123649597168 s
Within Limit: True
Norm to target: [11.68950003] mm

Reset took 0.03548860549926758 s
Within Limit: True
Norm t

KeyboardInterrupt: 

In [5]:
env.close()

In [2]:
import os
import numpy as np
import collections

import gym
import mujoco as mj
from mujoco.glfw import glfw

from pathlib import Path


# TARGETS_FILE = Path(__file__).parent.parent/"darm_targets.npy REF_POSE TODO"
DARM_XML_FILE = f"{os.getenv('DARM_MUJOCO_PATH')}/mujoco_env/darm.xml"

In [3]:
mj_model = mj.MjModel.from_xml_path(DARM_XML_FILE)

if mj_model: 
    print("Loaded XML file successfully") 
    mj_data = mj.MjData(mj_model)
else:
    print(f"Error Loading XML file: {xml_path}")

Loaded XML file successfully


In [4]:
mj_model.nu, mj_model.njnt

(5, 4)

In [31]:
mj_model.jnt_names

AttributeError: 'mujoco._structs.MjModel' object has no attribute 'jnt_names'

In [4]:
import math

min_joint_vals = []
max_joint_vals = []

for i in range(mj_model.njnt):
    joint_range = mj_model.jnt_range[i]*(180/np.pi)
    min_joint_vals.append(joint_range[0])
    max_joint_vals.append(joint_range[1])   
    
# min_joint_vals, max_joint_vals

In [5]:
def get_joint_limits(type = None):
    joint_limits = []
    for i in range(mj_model.njnt):
        joint_limits.append(mj_model.jnt_range[i]*(180/np.pi))
    
    joint_limits = np.asarray(joint_limits)
    
    if type == "min":
        return joint_limits[:, 0]
    if type == "max":
        return joint_limits[:, 1]
    
    return joint_limits[:, 0], joint_limits[:, 1]
    

In [6]:
def get_actuator_ctrlrange(type = None):
    if type == "min":
        return np.array([mj_model.actuator_ctrlrange[i][0] for i in range(mj_model.nu)])
    if type == "max":
        return np.array([mj_model.actuator_ctrlrange[i][1] for i in range(mj_model.nu)])
    
    ctrl_range = np.array([mj_model.actuator_ctrlrange[i] for i in range(mj_model.nu)])
    return ctrl_range[:, 0], ctrl_range[:, 1] # (min, max)
    

In [7]:
def compute_target_joint_state_delta(min_vals, max_vals):
    # return (max_vals - min_vals)//10  # for every range of 10 deg, have a delta of 1 deg
    return ((max_vals - min_vals)//40) * 2  # for every range of 40 deg, have a delta of 2 deg

In [8]:
get_joint_limits()

(array([-20., -45., -10., -10., -20., -45., -10., -10., -20., -45., -10.,
        -10., -50., -15., -20., -10., -60., -10., -10.,  -5., -20., -45.,
        -10., -10.]),
 array([ 20.,  90.,  90.,  90.,  20.,  90.,  90.,  90.,  20.,  90.,  90.,
         90., -10.,  15.,  20.,  90.,  90.,  10.,  10.,   5.,  20.,  90.,
         90.,  90.]))

In [9]:
get_actuator_ctrlrange("max")

array([ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
       10.,  5.,  5.,  5.,  5., 10.,  5.,  5.,  5.,  5., 10.,  5.,  5.,
        5.,  5., 10.,  5.])

In [10]:
compute_target_joint_state_delta(*get_joint_limits())

array([2., 6., 4., 4., 2., 6., 4., 4., 2., 6., 4., 4., 2., 0., 2., 4., 6.,
       0., 0., 0., 2., 6., 4., 4.])

In [11]:
np.clip(compute_target_joint_state_delta(*get_joint_limits()), a_min=2, a_max=np.inf)  # max =10

array([2., 6., 4., 4., 2., 6., 4., 4., 2., 6., 4., 4., 2., 2., 2., 4., 6.,
       2., 2., 2., 2., 6., 4., 4.])

In [12]:
min_vals, max_vals = get_joint_limits()
((max_vals - min_vals)//40) * 4 # for every range of 40 deg, have a delta of 4 deg

array([ 4., 12.,  8.,  8.,  4., 12.,  8.,  8.,  4., 12.,  8.,  8.,  4.,
        0.,  4.,  8., 12.,  0.,  0.,  0.,  4., 12.,  8.,  8.])

In [11]:
np.pi, math.pi

(3.141592653589793, 3.141592653589793)

In [12]:
min_vals = None

In [13]:
a = min_vals or [1]
a

[1]

In [14]:
np.array([10, 30, 5])/np.array([20, 30, 100]), 5/100

(array([0.5 , 1.  , 0.05]), 0.05)

In [15]:
a = np.array([1,2,3,4])
c = np.array([1,2,3,4])
b = np.array([1,2,3,4])

np.array([a, b, c]).shape, np.sum([a, b, c], axis=-2)
np.linalg.norm([4.5, 4.5, 4.5, 4.5, 5, 5])/np.sqrt(6/3)

8.093207028119323

In [16]:
((4.5**10) + (4.5**10) + (4.5**10))**(1/10)

5.02255428315257

In [39]:
time_prev = mj_data.time
action_time = 0.08
mj_data.ctrl = np.random.uniform(low=np.ones(mj_model.nu), high=np.ones(mj_model.nu)*5)

while True: # (mj_data.time - time_prev < action_time):
    mj.mj_step(mj_model, mj_data)
    if len(mj_data.contact.geom1) > 0:
        break
    
mj_data.time

0.02800000000000001

In [19]:
get_joint_limits("min")

array([-20., -45., -10., -10., -20., -45., -10., -10., -20., -45., -10.,
       -10., -50., -15., -20., -10., -60., -10., -10.,  -5., -20., -45.,
       -10., -10.])

In [7]:
mj_data.qpos = ((get_joint_limits("max") - get_joint_limits("min"))/2 + 0) * np.pi/180
mj.mj_forward(mj_model, mj_data)
len(mj_data.contact.geom1)

0

In [9]:
mj_model.nbody

33

In [20]:
mj_data.contact

<_MjContactList
  H: array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        

In [67]:
mj_data.cacc

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0.

In [22]:
# np.zeros((10,3)) + np.ones(3)
np.array([])

array([], dtype=float64)

In [23]:
space = gym.spaces.Box(low=np.array([-1.0]*4), 
                                            high=np.array([1.0]*4), 
                                            shape=(4,), dtype=np.float32)

/home/daniel/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [68]:
200*0.08

16.0

In [26]:
np.array([-0.01383569]) + 4*np.array([0.]) + 50*np.array([-0.]) + 0.1*-0.3296048641656354

array([-0.04679618])